In [1]:
import json

In [2]:
porter_data = json.load(open('terms_porter.json'))
mystem_data = json.load(open('terms_mystem.json'))

#### Первый параметр - список термом по портеру или mystem.
#### Второй - поисковый запрос из термов

In [3]:
def search(data, query):
    
    terms = query.split(' ')
    
    negative_terms = list(filter(lambda t : t.startswith('-'), terms))
    positive_terms = list(filter(lambda t : not t.startswith('-'), terms))
    
    positive_terms_amount = []
    for term in positive_terms:
        positive_terms_amount.append(data[term]['amount'])
        
    positive_terms_with_amount_zipped = list(zip(positive_terms_amount, positive_terms))
    positive_terms_with_amount_zipped.sort()
    
    positive_terms = [term for amount, term in positive_terms_with_amount_zipped]
        
    positive_docs = set(data[positive_terms[0]]['docs'])
    
    for term in positive_terms:
        positive_docs &= set(data[term]['docs'])
        
    for term in negative_terms:
        for doc in set(data[term[1:]]['docs']):
            positive_docs.discard(doc)
        
    return positive_docs

In [4]:
search(porter_data, 'в вид граничн')

{'http://www.mathnet.ru/rus/uzku1349'}

In [5]:
search(mystem_data, 'в вид')

{'http://www.mathnet.ru/rus/uzku1348', 'http://www.mathnet.ru/rus/uzku1349'}

In [6]:
search(mystem_data, '-в вид')

{'http://www.mathnet.ru/rus/uzku1354'}